In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
# Shell Plus Model Imports
from django.contrib.admin.models import LogEntry
from django.contrib.auth.models import Group, Permission, User
from django.contrib.contenttypes.models import ContentType
from django.contrib.sessions.models import Session
# Shell Plus Django Imports
from django.core.cache import cache
from django.conf import settings
from django.contrib.auth import get_user_model
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When, Exists, OuterRef, Subquery
from django.utils import timezone
from django.urls import reverse

Using development settings


FileNotFoundError: [Errno 2] No such file or directory: 'dj_secret.key'

Drug Inventory Models
- cmsinv.InventoryItem -> inventory_item
    - id, stock_qty

In [3]:
from cmsinv.models import InventoryItem

In [4]:
drugs = InventoryItem.objects.all()

In [5]:
count = 0
drugs[0]

<InventoryItem: HK42684 | 0.05% CHLORHEXIDINE & 0.5% CETRIMIDE SOL / 0.05% CHLORHEXIDINE & 0.5% CETRIMIDE SOL [0.05% CHLORHEXIDINE & 0.5% CETRIMIDE SOL]>

In [14]:
used_drugs = InventoryItem.objects.filter(updated_by__isnull=False).exclude(updated_by='cmsman')

In [15]:
used_drugs.count()

639

In [16]:
count = 1
for drug in used_drugs:
    print(f"{count:<5}: {drug.updated_by:<8} | Qty: {drug.stock_qty:<8} | {drug.product_name}")
    count += 1

1    : RNURSE   | Qty: 9999.0   | 0.05% CHLORHEXIDINE & 0.5% CETRIMIDE SOL
2    : admin    | Qty: 10000.0  | ABC PLUS TAB
3    : sychanam | Qty: 53.0     | ACERTIL PLUS TAB 2.5 MG/0.625MG
4    : rnurse   | Qty: 1910.0   | ACETAMOL 250MG
5    : rnurse   | Qty: 3506.0   | ACETAMOL (G) TAB 300MG
6    : admin    | Qty: 155.0    | ZOVIRAX 200MG
7    : admin    | Qty: 235.0    | ZOVIRAX 400MG
8    : rnurse   | Qty: 5.0      | ACYVIR CREAM 5% W/W
9    : sychanam | Qty: 195.0    | ACYVIR TAB 800MG
10   : rnurse   | Qty: 0.0      | ADACEL POLIO VACCINE
11   : rnurse   | Qty: 0.0      | ADACEL VACCINE
12   : sychanam | Qty: 880.0    | AIRROX CHEWABLE TAB 80 MG
13   : RNURSE   | Qty: 1.0      | ALLNOL PLUS TAB 300MG
14   : NURSE    | Qty: 9904.0   | ALLNOL TAB 100MG
15   : RNURSE   | Qty: 516.0    | ALLOPURINOL TAB 300MG 
16   : rnurse   | Qty: 602.0    | ALLUMAG MPS TAB
17   : sychanam | Qty: 9169.0   | CLARITYNE SYRUP
18   : admin    | Qty: 10000.0  | AMITRIPTYLINE TAB 10MG (ALPHARMA)
19   : RN

In [9]:
drugs_to_reset = InventoryItem.objects.filter(Q(updated_by__isnull=True) | Q(updated_by=''))

In [11]:
count = 0
count10k = 0
reset_ids = []
for drug in drugs_to_reset:
    if drug.last_updated is not None:
        print(f"{drug.id:<6} last updated on {drug.last_updated}")
    if drug.discontinue == 0:
        # Set inactive (discontinue=1)
        print(f"{drug.id:<6} Mstr {drug.is_master_drug_list} Clin {drug.is_clinic_drug_list} | Set inactive: {drug.product_name}")
    if drug.stock_qty == 10000:
        # Reset to 0
        count10k += 1
        reset_ids.append(drug.id)
    elif drug.stock_qty == 0:
        print(f"Qty already 0: {drug.product_name}")
    else:
        print(f"[{stock_qty:<7}]: {drug.product_name}")
    count += 1
print(f"Counted: {count}; Counted stock_qty is 10k {count10k}")
print(reset_ids)

355    Mstr True Clin True | Set inactive: ADRIEN GAGNON CALCIUM MAGNESIUM WITH VITAMIN D AND ZINC TAB
938    Mstr True Clin True | Set inactive: AMPRA CAP 20MG (ENTERIC-COATED PELLETS)
1122   Mstr True Clin True | Set inactive: ANUSOL OINT
1299   Mstr True Clin True | Set inactive: APO-METOPROLOL 100MG TAB
2183   Mstr True Clin True | Set inactive: BEPROSONE CREAM 0.05%
2546   Mstr True Clin True | Set inactive: BOGEN-50 S C TAB
2633   Mstr True Clin True | Set inactive: BROM-RAMINE CO. SYRUP (NEW FORMULA)
2673   Mstr True Clin True | Set inactive: BROMHEXINE TAB 8MG (VICKMANS)
2702   Mstr True Clin True | Set inactive: BRONCO-DM TAB
2741   Mstr True Clin True | Set inactive: BUCANIN SYRUP 3MG/5ML
2757   Mstr True Clin True | Set inactive: BUPOGESIC 200 TAB 200MG
3176   Mstr True Clin True | Set inactive: CECLOR FOR ORAL SUSP 125MG/5ML
3813   Mstr True Clin True | Set inactive: CLACIN PLUS TAB 500MG
3821   Mstr True Clin True | Set inactive: CLAMOVID 625 TAB 625MG
3822   Mstr True Cli

In [12]:
# Reset stock_qty to 0
for drug in drugs_to_reset:
    if drug.stock_qty == 10000:
        print(drug.registration_no, drug.stock_qty, drug.product_name, drug.clinic_drug_no, drug.date_created, drug.last_updated)
        drug.stock_qty = 0
        drug.is_master_drug_list = False
        drug.is_clinic_drug_list = False
        drug.discontinue = True
        
        drug.save()

HK55323 10000.0 ADRIEN GAGNON CALCIUM MAGNESIUM WITH VITAMIN D AND ZINC TAB None None None
HK51086 10000.0 AMPRA CAP 20MG (ENTERIC-COATED PELLETS) None None None
HK06139 10000.0 ANUSOL OINT None None None
HK36923 10000.0 APO-METOPROLOL 100MG TAB None None None
HK45778 10000.0 BEPROSONE CREAM 0.05% None None None
HK17385 10000.0 BOGEN-50 S C TAB None None None
HK48604 10000.0 BROM-RAMINE CO. SYRUP (NEW FORMULA) None None None
HK20057 10000.0 BROMHEXINE TAB 8MG (VICKMANS) None None None
HK39022 10000.0 BRONCO-DM TAB None None None
HK39174 10000.0 BUCANIN SYRUP 3MG/5ML None None None
HK43233 10000.0 BUPOGESIC 200 TAB 200MG None None None
HK20158 10000.0 CECLOR FOR ORAL SUSP 125MG/5ML None None None
HK54294 10000.0 CLACIN PLUS TAB 500MG None None None
HK50621 10000.0 CLAMOVID 625 TAB 625MG None None None
HK50620 10000.0 CLAMOVID GRANULES 156.25MG/5ML None None None
HK17506 10000.0 COLDEE TAB None None None
HK46922 10000.0 CP-B CO CAPSULE None None None
HK44221 10000.0 DIFENOL GEL 1% None N

In [13]:
check = InventoryItem.objects.get(id=2)
print(check.registration_no, check.stock_qty, check.product_name, check.clinic_drug_no, check.date_created, check.last_updated)

HK42683 0.0 0.1% CHLORHEXIDINE & 1% CETRIMIDE SOLN None None None


In [30]:
check.stock_qty = 10000
print(check.registration_no, check.stock_qty, check.product_name, check.clinic_drug_no, check.date_created, check.last_updated)

HK42683 10000 0.1% CHLORHEXIDINE & 1% CETRIMIDE SOLN None None None


In [31]:
check.save()